# Python 資料清理 with Pandas

## 載入 Pandas 套件

In [ ]:
import pandas as pd

## 讀取 Excel 檔案，匯入為 Pandas DataFrame

In [ ]:
df = pd.read_excel('sample4.xlsx', sheet_name=0)

看一下讀進來的資料長得什麼樣子：

In [ ]:
df

看一下每個欄位是否有缺漏的資料：

In [ ]:
df.isnull().sum()

## 填補缺漏資料

我們先來處理「入學管道」與「報名日期」兩個欄位，我們讓這兩個欄位的空值直接填入前一筆資料的內容：

In [ ]:
df['入學管道'].fillna(method='ffill', inplace=True)
df['報名日期'].fillna(method='ffill', inplace=True)

接著處理「出生地」欄位，我們將出生地的空值䩗入「地址」欄位的前三個字：

In [ ]:
df['出生地'].fillna(df['地址'].str[:3], inplace=True)

再來處理「數學」與「自然」兩個欄位的缺值，「數學」欄位的缺值填0：

In [ ]:
df['數學'].fillna(0, inplace=True)

「自然」㯗位要填平均值：

In [ ]:
df['自然'].fillna(df['自然'].mean(), inplace=True)

## 移除重複的列

接下來把同樣學號與姓名的資料有重複的刪除：

In [ ]:
df.drop_duplicates(['學號', '姓名'], inplace=True)

## 變更大小寫

利用Python字串處理，把英文名字首字改為大寫：

In [ ]:
df['英文名'] = df['英文名'].str.title()

## 移除空白字元

一樣利用Python字串處理，把前後的空白移除：

In [ ]:
df['英文名'] = df['英文名'].str.strip()

## 修正數值格式

我們把「社會」、「數學」和「自然」的成績分數都四捨五入到整數位：

In [ ]:
df['社會'] = df['社會'].round(decimals=0).apply(int)
df['數學'] = df['數學'].round(decimals=0).apply(int)
df['自然'] = df['自然'].round(decimals=0).apply(int)

## 將「總分」與「平均」重新計算

我們把原本的「總分」與「平均」兩個欄位，按照新的值重新來計算：

In [ ]:
df['總分'] = df.loc[:,['國文', '英文', '數學', '社會', '自然']].sum(axis=1)
df['平均'] = df.loc[:,['國文', '英文', '數學', '社會', '自然']].mean(axis=1)

In [ ]:
dfm

## 修正日期

將「生日」欄位的日期統一為西元年並都用同樣的格式：

In [ ]:
def convert_date(x):
    x = x.replace('-', '/')
    y, m, d = x.split('/')
    y, m, d = int(y), int(m), int(d)
    return f"{y + 1911}/{m:02d}/{d:02d}"
    
df['生日'] = df['生日'].apply(convert_date)

## 電話欄位格式

我們利用正規表示式來統一電話格式為 **02**-XXXXXXX：

In [ ]:
import re
import numpy as np

def convert_phone(x):
    if pd.notnull(x):
        m = re.match('\((.*)\)(.*)', x) 
        if m:
            area_code = m.group(1)
            last_part = m.group(2)
            return f"{area_code}-{last_part}"
        else:
            return x
    else:
        return np.NaN
    
df['電話'].apply(convert_phone)

雖然這個在Excel看起來似乎也做得到，但若我們要求是要統一成這樣： **(02)** XXXX-XXXX 話的，Excel就很難辦到了，但是用Python簡單的修改一下就可以：

In [ ]:
import re
import numpy as np

def convert_phone(x):
    if pd.notnull(x):
        m = re.match('(\d\d)-(.*)', x) 
        if m:
            area_code = m.group(1)
            last_part = m.group(2)
            return f"({area_code}){last_part}"
        else:
            return x
    else:
        return np.NaN
    
df['電話'].apply(convert_phone)

In [ ]:
df['電話'] = df['電話'].apply(convert_phone)

## 清理完成！

我們來看一下好不容易清理完的結果吧：

In [ ]:
df

## 把結果輸出成 Excel 檔案：

In [ ]:
df.to_excel('cleaned.xlsx', index=False)